In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_LRdown_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_LRdown_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4] + "_LRdown"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_1220\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 130.71 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX5
            dataY = dataY5
            lr = 0.0032
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            lr = 0.0016
        else:
            #5s
            dataX = dataX1
            dataY = dataY1
            lr = 0.0008

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.8054087162017822 Train_acc: 2.437500%
Train Epoch: 2/60 Train_Loss: 3.630220890045166 Train_acc: 5.437500%
Train Epoch: 3/60 Train_Loss: 3.4501781463623047 Train_acc: 6.125000%
Train Epoch: 4/60 Train_Loss: 3.376631021499634 Train_acc: 7.062500%
Train Epoch: 5/60 Train_Loss: 3.355865955352783 Train_acc: 8.562500%
Train Epoch: 6/60 Train_Loss: 3.02071475982666 Train_acc: 13.125000%
Train Epoch: 7/60 Train_Loss: 3.2330470085144043 Train_acc: 13.562500%
Train Epoch: 8/60 Train_Loss: 3.083766460418701 Train_acc: 12.312500%
Train Epoch: 9/60 Train_Loss: 2.854501247406006 Train_acc: 15.750000%
Train Epoch: 10/60 Train_Loss: 2.9485983848571777 Train_acc: 19.000000%
Train Epoch: 11/60 Train_Loss: 2.775384902954

Train Epoch: 47/60 Train_Loss: 0.6500876545906067 Train_acc: 77.145836%
Train Epoch: 48/60 Train_Loss: 0.8173734545707703 Train_acc: 77.854172%
Train Epoch: 49/60 Train_Loss: 0.5756948590278625 Train_acc: 77.541672%
Train Epoch: 50/60 Train_Loss: 0.8823383450508118 Train_acc: 78.104172%
Train Epoch: 51/60 Train_Loss: 0.7098405957221985 Train_acc: 77.520836%
Train Epoch: 52/60 Train_Loss: 0.7045508623123169 Train_acc: 77.625000%
Train Epoch: 53/60 Train_Loss: 1.0300250053405762 Train_acc: 77.791672%
Train Epoch: 54/60 Train_Loss: 0.8813081979751587 Train_acc: 77.645836%
Train Epoch: 55/60 Train_Loss: 0.6593878865242004 Train_acc: 77.958336%
Train Epoch: 56/60 Train_Loss: 0.5699940323829651 Train_acc: 77.937500%
Train Epoch: 57/60 Train_Loss: 0.5662247538566589 Train_acc: 77.833336%
Train Epoch: 58/60 Train_Loss: 0.6835504174232483 Train_acc: 78.791672%
Train Epoch: 59/60 Train_Loss: 0.806218683719635 Train_acc: 78.854172%
Train Epoch: 60/60 Train_Loss: 0.7400885820388794 Train_acc: 78.8

Train Epoch: 31/60 Train_Loss: 1.656274437904358 Train_acc: 41.437500%
Train Epoch: 32/60 Train_Loss: 1.8116440773010254 Train_acc: 41.687500%
Train Epoch: 33/60 Train_Loss: 1.7317196130752563 Train_acc: 43.000000%
Train Epoch: 34/60 Train_Loss: 1.7008990049362183 Train_acc: 42.562500%
Train Epoch: 35/60 Train_Loss: 1.890708565711975 Train_acc: 44.312500%
Train Epoch: 36/60 Train_Loss: 2.0647149085998535 Train_acc: 43.250000%
Train Epoch: 37/60 Train_Loss: 1.6728359460830688 Train_acc: 44.000000%
Train Epoch: 38/60 Train_Loss: 1.9140899181365967 Train_acc: 43.625000%
Train Epoch: 39/60 Train_Loss: 1.6241049766540527 Train_acc: 44.750000%
Train Epoch: 40/60 Train_Loss: 1.7147252559661865 Train_acc: 44.437500%
Train Epoch: 41/60 Train_Loss: 1.9368045330047607 Train_acc: 46.312500%
Train Epoch: 42/60 Train_Loss: 1.6091060638427734 Train_acc: 47.625000%
Train Epoch: 43/60 Train_Loss: 1.5903741121292114 Train_acc: 46.687500%
Train Epoch: 44/60 Train_Loss: 1.7556414604187012 Train_acc: 47.50

Train Epoch: 15/60 Train_Loss: 1.7220629453659058 Train_acc: 57.775002%
Train Epoch: 16/60 Train_Loss: 1.2151530981063843 Train_acc: 59.525002%
Train Epoch: 17/60 Train_Loss: 1.6760802268981934 Train_acc: 60.750004%
Train Epoch: 18/60 Train_Loss: 1.4170786142349243 Train_acc: 59.950005%
Train Epoch: 19/60 Train_Loss: 1.5249689817428589 Train_acc: 60.975002%
Train Epoch: 20/60 Train_Loss: 1.1252925395965576 Train_acc: 60.700005%
Train Epoch: 21/60 Train_Loss: 1.4155153036117554 Train_acc: 60.825005%
Train Epoch: 22/60 Train_Loss: 1.4125841856002808 Train_acc: 61.012505%
Train Epoch: 23/60 Train_Loss: 1.3442553281784058 Train_acc: 61.737503%
Train Epoch: 24/60 Train_Loss: 1.2859654426574707 Train_acc: 61.537502%
Train Epoch: 25/60 Train_Loss: 1.4249963760375977 Train_acc: 62.087502%
Train Epoch: 26/60 Train_Loss: 1.1279363632202148 Train_acc: 61.662502%
Train Epoch: 27/60 Train_Loss: 1.0953457355499268 Train_acc: 61.437504%
Train Epoch: 28/60 Train_Loss: 1.295554518699646 Train_acc: 62.2

Train Epoch: 1/60 Train_Loss: 1.6652610301971436 Train_acc: 41.479168%
Train Epoch: 2/60 Train_Loss: 1.5549501180648804 Train_acc: 51.270836%
Train Epoch: 3/60 Train_Loss: 1.4171380996704102 Train_acc: 53.437500%
Train Epoch: 4/60 Train_Loss: 1.411544680595398 Train_acc: 54.062500%
Train Epoch: 5/60 Train_Loss: 1.380311131477356 Train_acc: 54.458336%
Train Epoch: 6/60 Train_Loss: 1.5574828386306763 Train_acc: 54.187500%
Train Epoch: 7/60 Train_Loss: 1.2952899932861328 Train_acc: 56.395836%
Train Epoch: 8/60 Train_Loss: 1.241086483001709 Train_acc: 56.500000%
Train Epoch: 9/60 Train_Loss: 1.293667197227478 Train_acc: 57.729168%
Train Epoch: 10/60 Train_Loss: 1.4660392999649048 Train_acc: 59.437500%
Train Epoch: 11/60 Train_Loss: 1.290421724319458 Train_acc: 58.625000%
Train Epoch: 12/60 Train_Loss: 1.3972928524017334 Train_acc: 59.895836%
Train Epoch: 13/60 Train_Loss: 1.0238947868347168 Train_acc: 61.812500%
Train Epoch: 14/60 Train_Loss: 1.3183568716049194 Train_acc: 60.479168%
Train 

Train Epoch: 50/60 Train_Loss: 0.9569191932678223 Train_acc: 72.812500%
Train Epoch: 51/60 Train_Loss: 0.6187646985054016 Train_acc: 72.637505%
Train Epoch: 52/60 Train_Loss: 0.9510771632194519 Train_acc: 72.450005%
Train Epoch: 53/60 Train_Loss: 1.0288375616073608 Train_acc: 72.562500%
Train Epoch: 54/60 Train_Loss: 0.9469990134239197 Train_acc: 72.600006%
Train Epoch: 55/60 Train_Loss: 1.1019231081008911 Train_acc: 72.937500%
Train Epoch: 56/60 Train_Loss: 0.773273229598999 Train_acc: 72.737503%
Train Epoch: 57/60 Train_Loss: 0.9175853729248047 Train_acc: 73.062500%
Train Epoch: 58/60 Train_Loss: 1.163870096206665 Train_acc: 72.987503%
Train Epoch: 59/60 Train_Loss: 0.924207329750061 Train_acc: 73.025002%
Train Epoch: 60/60 Train_Loss: 0.7537204027175903 Train_acc: 72.612503%
Test accuracy of the model: 39.100000%
Test accuracy of the model: 44.666667%
Test accuracy of the model: 43.750000%
Training Time: 53.04 seconds
可用的 GPU 数量: 1
---------------------------------------------------

Train Epoch: 34/60 Train_Loss: 0.5169763565063477 Train_acc: 76.479172%
Train Epoch: 35/60 Train_Loss: 0.8392115831375122 Train_acc: 76.708336%
Train Epoch: 36/60 Train_Loss: 0.7276941537857056 Train_acc: 76.770836%
Train Epoch: 37/60 Train_Loss: 0.707809567451477 Train_acc: 77.250000%
Train Epoch: 38/60 Train_Loss: 0.543753981590271 Train_acc: 77.541672%
Train Epoch: 39/60 Train_Loss: 0.6915836930274963 Train_acc: 77.187500%
Train Epoch: 40/60 Train_Loss: 0.3775419592857361 Train_acc: 78.229172%
Train Epoch: 41/60 Train_Loss: 0.9100604057312012 Train_acc: 78.354172%
Train Epoch: 42/60 Train_Loss: 0.5403603911399841 Train_acc: 78.312500%
Train Epoch: 43/60 Train_Loss: 0.8130552172660828 Train_acc: 77.562500%
Train Epoch: 44/60 Train_Loss: 0.5820392966270447 Train_acc: 79.000000%
Train Epoch: 45/60 Train_Loss: 0.8444328904151917 Train_acc: 78.250000%
Train Epoch: 46/60 Train_Loss: 0.5397669672966003 Train_acc: 80.916672%
Train Epoch: 47/60 Train_Loss: 0.7770649790763855 Train_acc: 81.29

Train Epoch: 18/60 Train_Loss: 2.1947219371795654 Train_acc: 31.187500%
Train Epoch: 19/60 Train_Loss: 2.0441508293151855 Train_acc: 32.812500%
Train Epoch: 20/60 Train_Loss: 1.9684629440307617 Train_acc: 33.875000%
Train Epoch: 21/60 Train_Loss: 2.277127742767334 Train_acc: 35.500000%
Train Epoch: 22/60 Train_Loss: 2.2821438312530518 Train_acc: 37.500000%
Train Epoch: 23/60 Train_Loss: 2.057691812515259 Train_acc: 34.687500%
Train Epoch: 24/60 Train_Loss: 1.8419506549835205 Train_acc: 37.437500%
Train Epoch: 25/60 Train_Loss: 2.0679845809936523 Train_acc: 35.500000%
Train Epoch: 26/60 Train_Loss: 1.9640411138534546 Train_acc: 33.437500%
Train Epoch: 27/60 Train_Loss: 1.8911664485931396 Train_acc: 38.187500%
Train Epoch: 28/60 Train_Loss: 1.9159190654754639 Train_acc: 38.875000%
Train Epoch: 29/60 Train_Loss: 1.9000482559204102 Train_acc: 40.750000%
Train Epoch: 30/60 Train_Loss: 1.5348434448242188 Train_acc: 41.500000%
Train Epoch: 31/60 Train_Loss: 1.5887939929962158 Train_acc: 44.50

Train Epoch: 2/60 Train_Loss: 1.6822117567062378 Train_acc: 48.575001%
Train Epoch: 3/60 Train_Loss: 1.3596476316452026 Train_acc: 50.275002%
Train Epoch: 4/60 Train_Loss: 1.4332945346832275 Train_acc: 51.937504%
Train Epoch: 5/60 Train_Loss: 2.22290301322937 Train_acc: 52.075001%
Train Epoch: 6/60 Train_Loss: 1.6911773681640625 Train_acc: 53.550003%
Train Epoch: 7/60 Train_Loss: 1.2777559757232666 Train_acc: 55.400002%
Train Epoch: 8/60 Train_Loss: 1.5065054893493652 Train_acc: 54.762501%
Train Epoch: 9/60 Train_Loss: 1.7126542329788208 Train_acc: 54.762501%
Train Epoch: 10/60 Train_Loss: 1.657468318939209 Train_acc: 56.450005%
Train Epoch: 11/60 Train_Loss: 1.4209073781967163 Train_acc: 57.025002%
Train Epoch: 12/60 Train_Loss: 1.6547505855560303 Train_acc: 57.875004%
Train Epoch: 13/60 Train_Loss: 1.1200495958328247 Train_acc: 57.912502%
Train Epoch: 14/60 Train_Loss: 1.4216727018356323 Train_acc: 58.062504%
Train Epoch: 15/60 Train_Loss: 1.1358509063720703 Train_acc: 58.700005%
Tra

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()